<h1><font color='blue'> 8E and 8F: Finding the Probability P(Y==1|X)</font></h1>

<h2><font color='Geen'> 8E: Implementing Decision Function of SVM RBF Kernel</font></h2>

<font face=' Comic Sans MS' size=3>After we train a kernel SVM model, we will be getting support vectors and their corresponsing coefficients $\alpha_{i}$

Check the documentation for better understanding of these attributes: 

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
<img src='https://i.imgur.com/K11msU4.png' width=500>

As a part of this assignment you will be implementing the ```decision_function()``` of kernel SVM, here decision_function() means based on the value return by ```decision_function()``` model will classify the data point either as positive or negative

Ex 1: In logistic regression After traning the models with the optimal weights $w$ we get, we will find the value $\frac{1}{1+\exp(-(wx+b))}$, if this value comes out to be < 0.5 we will mark it as negative class, else its positive class

Ex 2: In Linear SVM After traning the models with the optimal weights $w$ we get, we will find the value of $sign(wx+b)$, if this value comes out to be -ve we will mark it as negative class, else its positive class.

Similarly in Kernel SVM After traning the models with the coefficients $\alpha_{i}$ we get, we will find the value of 
$sign(\sum_{i=1}^{n}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here $K(x_{i},x_{q})$ is the RBF kernel. If this value comes out to be -ve we will mark $x_{q}$ as negative class, else its positive class.

RBF kernel is defined as: $K(x_{i},x_{q})$ = $exp(-\gamma ||x_{i} - x_{q}||^2)$

For better understanding check this link: https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation
</font>

## Task E

> 1. Split the data into $X_{train}$(60), $X_{cv}$(20), $X_{test}$(20)

> 2. Train $SVC(gamma=0.001, C=100.)$ on the ($X_{train}$, $y_{train}$)

> 3. Get the decision boundry values $f_{cv}$ on the $X_{cv}$ data  i.e. ` `$f_{cv}$ ```= decision_function(```$X_{cv}$```)```  <font color='red'>you need to implement this decision_function()</font>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import numpy as np
from sklearn.svm import SVC

In [2]:
from sklearn.model_selection import train_test_split
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train)
X_train.shape, X_cv.shape, X_test.shape


((3000, 5), (1000, 5), (1000, 5))

### Pseudo code

clf = SVC(gamma=0.001, C=100.)<br>
clf.fit(Xtrain, ytrain)

<font color='green'>def</font> <font color='blue'>decision_function</font>(Xcv, ...): #use appropriate parameters <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'>for</font> a data point $x_q$ <font color='green'>in</font> Xcv: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='grey'>#write code to implement $(\sum_{i=1}^{\text{all the support vectors}}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here the values $y_i$, $\alpha_{i}$, and $intercept$ can be obtained from the trained model</font><br>
   <font color='green'>return</font> <font color='grey'><i># the decision_function output for all the data points in the Xcv</i></font>
    
fcv = decision_function(Xcv, ...)  <i># based on your requirement you can pass any other parameters </i>

<b>Note</b>: Make sure the values you get as fcv, should be equal to outputs of clf.decision_function(Xcv)


In [4]:
# you can write your code here
from sklearn.svm import SVC
import math

def decision_function(X_cv,clf):
  cv_desc_func = []
  for x_q in X_cv:
        decision_x_q = np.sum(clf.dual_coef_*np.exp(-clf._gamma*np.sum((clf.support_vectors_-x_q)**2,axis=1)))+clf.intercept_
        cv_desc_func.append(decision_x_q)
  return np.array(cv_desc_func)

gamma_val = 0.001
clf = SVC(C=100, kernel='rbf', gamma=gamma_val)
clf.fit(X_train,y_train)
decis_func = clf.decision_function(X_cv)
print(decis_func[:5])
decis_func_calc = decision_function(X_cv,clf)
print(decis_func_calc[:5])


[-0.74439228  1.77714849  1.64888727  1.52845213  0.55574548]
[[-0.74439228]
 [ 1.77714849]
 [ 1.64888727]
 [ 1.52845213]
 [ 0.55574548]]


<h2><font color='Geen'> 8F: Implementing Platt Scaling to find P(Y==1|X)</font></h2>

Check this <a href='https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a'>PDF</a>
<img src='https://i.imgur.com/CAMnVnh.png'>


## TASK F


> 4. Apply SGD algorithm with ($f_{cv}$, $y_{cv}$) and find the weight $W$ intercept $b$ ```Note: here our data is of one dimensional so we will have a one dimensional weight vector i.e W.shape (1,)``` 

> Note1: Don't forget to change the values of $y_{cv}$ as mentioned in the above image. you will calculate y+, y- based on data points in train data

> Note2: the Sklearn's SGD algorithm doesn't support the real valued outputs, you need to use the code that was done in the `'Logistic Regression with SGD and L2'` Assignment after modifying loss function, and use same parameters that used in that assignment.
<img src='https://i.imgur.com/zKYE9Oc.png'>
if Y[i] is 1, it will be replaced with y+ value else it will replaced with y- value

> 5. For a given data point from $X_{test}$, $P(Y=1|X) = \frac{1}{1+exp(-(W*f_{test}+ b))}$ where ` `$f_{test}$ ```= decision_function(```$X_{test}$```)```, W and b will be learned as metioned in the above step

In [4]:
from sklearn import linear_model
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

def initialize_weights(dim):
    ''' In this function, we will initialize our weights and bias'''
    #initialize the weights to zeros array of (1,dim) dimensions
    #you use zeros_like function to initialize zero, check this link https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros_like.html
    #initialize bias to zero

    w = np.zeros_like(dim)
    b = 0
    
    return w,b

def sigmoid(z):
    ''' In this function, we will return sigmoid of z'''
    # compute sigmoid(z) and return
    sigmoid_z = 1/(1+math.exp(-1*z))
    return sigmoid_z

def gradient_dw(x,y,w,b,alpha,N):
    '''In this function, we will compute the gardient w.r.to w '''
    dw = x*(y-sigmoid(np.matmul(w.transpose(),x)+b))-(alpha/N)*w
    return dw

def gradient_db(x,y,w,b):
     '''In this function, we will compute gradient w.r.to b '''
     db = y-sigmoid(np.matmul(w.transpose(),x)+b) 
     return db

def pred(w,b, X):
    N = len(X)
    predict = []
    for i in range(N):
        z=np.dot(w,X[i])+b
        if sigmoid(z) >= 0.5: # sigmoid(w,x,b) returns 1/(1+exp(-(dot(x,w)+b)))
            predict.append(1)
        else:
            predict.append(0)
    return np.array(predict)

def logloss(y_true,y_pred):
    '''In this function, we will compute log loss '''
    temp = 0
    for i in range(len(y_true)):
      y_pred_float = y_pred.astype(np.float)
      if y_pred_float[i]<=0.0 or y_pred_float[i]==1.0:
        if y_pred_float[i]==0.0:
          y_pred_float[i] = 1e-12
        elif y_pred_float[i]==1.0:
          y_pred_float[i] = 1-1e-12
        else:
          y_pred_float[i] = abs(y_pred_float[i])
      temp = temp+((y_true[i]*math.log10(y_pred_float[i]))+((1-y_true[i])*math.log10(1-y_pred_float[i])))
    loss = -1*(temp/len(y_true))
    return loss

def train(X_train,y_train,X_test,y_test,epochs,alpha,eta0):
    ''' In this function, we will implement logistic regression'''
    #Here eta0 is learning rate
    #implement the code as follows
    # initalize the weights (call the initialize_weights(X_train[0]) function)
    # for every epoch
        # for every data point(X_train,y_train)
           #compute gradient w.r.to w (call the gradient_dw() function)
           #compute gradient w.r.to b (call the gradient_db() function)
           #update w, b
        # predict the output of x_train[for all data points in X_train] using w,b
        #compute the loss between predicted and actual values (call the loss function)
        # store all the train loss values in a list
        # predict the output of x_test[for all data points in X_test] using w,b
        #compute the loss between predicted and actual values (call the loss function)
        # store all the test loss values in a list
        # you can also compare previous loss and current loss, if loss is not updating then stop the process and return w,b
    w,b=initialize_weights(X_train[0])
    train_loss = []
    test_loss = []
    for j in tqdm(range(epochs)):
      for i in range(len(X_train)):
        dw = gradient_dw(X_train[i],y_train[i],w,b,alpha,len(X_train))
        db = gradient_db(X_train[i],y_train[i],w,b)
        w = w+eta0*dw
        b = b+eta0*db
      y_true_train = y_train
      y_pred_train = pred(w,b, X_train)  
      loss = logloss(y_true_train,y_pred_train)
      train_loss.append(loss)
      y_true_test = y_test
      y_pred_test = pred(w,b, X_test)  
      loss = logloss(y_true_test,y_pred_test)
      test_loss.append(loss)
    plt.plot(list(range(epochs)),train_loss,label="Train dataset loss") 
    plt.plot(list(range(epochs)),test_loss,label="Test dataset loss") 
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    return w,b

num_pos=0
num_neg=0
for i in range(len(y_train)):
    if y_train[i]>0:
        num_pos+=1
    else:
        num_neg+=1
y_pos = (num_pos+1)/(num_pos+2)
y_neg = 1/(num_neg+2)
y_cv = np.where(y_cv==1,y_pos,y_neg)

clf = linear_model.SGDClassifier(eta0=0.0001, alpha=0.0001, loss='log', random_state=15, penalty='l2', tol=1e-3, verbose=2, learning_rate='constant')
clf.fit(X_train, y_train)
f_cv = clf.decision_function(X_cv)
f_test = clf.decision_function(X_test)

alpha=0.0001
eta0=0.0001
N=len(f_cv)
epochs=14
w,b=train(f_cv,y_cv,f_test,y_test,epochs,alpha,eta0)
print(w)
print(b)


-- Epoch 1
Norm: 0.11, NNZs: 5, Bias: -0.055714, T: 3000, Avg. loss: 0.665312
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.22, NNZs: 5, Bias: -0.104952, T: 6000, Avg. loss: 0.615582
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.32, NNZs: 5, Bias: -0.148501, T: 9000, Avg. loss: 0.573666
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.41, NNZs: 5, Bias: -0.187073, T: 12000, Avg. loss: 0.538114
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.50, NNZs: 5, Bias: -0.221359, T: 15000, Avg. loss: 0.507724
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.58, NNZs: 5, Bias: -0.251881, T: 18000, Avg. loss: 0.481548
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.65, NNZs: 5, Bias: -0.279206, T: 21000, Avg. loss: 0.458816
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 0.72, NNZs: 5, Bias: -0.303773, T: 24000, Avg. loss: 0.438922
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 0.79, NNZs: 5, Bias: -0.325819, T: 27000, Avg. loss: 0.421393
Total traini

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

__Note: in the above algorithm, the steps 2, 4 might need hyper parameter tuning, To reduce the complexity of the assignment we are excluding the hyerparameter tuning part, but intrested students can try that__


If any one wants to try other calibration algorithm istonic regression also please check these tutorials

1. http://fa.bianp.net/blog/tag/scikit-learn.html#fn:1

2. https://drive.google.com/open?id=1MzmA7QaP58RDzocB0RBmRiWfl7Co_VJ7

3. https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a

4. https://stat.fandom.com/wiki/Isotonic_regression#Pool_Adjacent_Violators_Algorithm
